In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
video_path = '/content/drive/MyDrive/CountingUsingYolov8Workshop/videocars.mp4'  # Update this with the correct file name if different

In [ ]:
!pip install ultralytics
!pip install opencv-python
!pip install filterpy==1.4.5

In [ ]:
# Download the sort.py file
!wget https://raw.githubusercontent.com/abewley/sort/master/sort.py

# Modify the sort.py file to remove matplotlib dependency
with open('sort.py', 'r') as file:
    lines = file.readlines()

# Remove or comment out any lines that import or use matplotlib
new_lines = []
for line in lines:
    if "import matplotlib" not in line and "matplotlib.use" not in line:
        new_lines.append(line)

with open('sort.py', 'w') as file:
    file.writelines(new_lines)

In [ ]:
# Import required libraries
import cv2
import numpy as np
from ultralytics import YOLO
from sort import Sort  # Ensure the correct import for SORT
from google.colab import drive  # To mount Google Drive
from google.colab.patches import cv2_imshow  # To display frames in Colab

In [ ]:
# Define paths
output_folder = '/content/drive/MyDrive/CountingUsingYolov8Workshop/output'  # Output folder in Google Drive
output_video_path = f'{output_folder}/output_video_cars.mp4'  # Path to save the output video

In [ ]:
# Load the YOLO model
model = YOLO('yolov8m.pt')  # Load a pretrained model

# Define the line for counting vehicles
LINE = [300, 300, 750, 300]

# Initialize the tracker
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# List to store unique vehicle IDs
totalCount = []

# Open the video file
vid = cv2.VideoCapture(video_path)

# Get video properties
fps = int(vid.get(cv2.CAP_PROP_FPS))
frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = vid.read()
    if not ret:
        break  # End of video or failed read

    # Detect objects in the frame
    results = model(frame, stream=True)
    detections = np.empty((0, 5))

    # Filter detections for vehicles (car, motorcycle, truck, bus)
    for r in results:
        boxes = r.boxes
        for bbox in boxes:
            x1, y1, x2, y2 = bbox.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cls_idx = int(bbox.cls[0])  # Get the class index
            cls_name = model.names[cls_idx]  # Get the class name

            conf = round(float(bbox.conf[0]), 2)  # Get the confidence score

            if cls_name in ['car', 'motorcycle', 'truck', 'bus']:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (225, 0, 0), 3)
                cv2.putText(frame, f'{cls_name} {conf}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0))

                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Update the tracker with the current detections
    resultsTracker = tracker.update(detections)

    # Process each tracked vehicle
    for result in resultsTracker:
        x1, y1, x2, y2, ID = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Calculate the center of the bounding box
        w, h = x2 - x1, y2 - y1
        cx = x1 + w // 2
        cy = y1 + h // 2

        # Draw the center and bounding box
        cv2.circle(frame, (cx, cy), radius=4, color=(255, 255, 0), thickness=cv2.FILLED)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (225, 0, 0), 3)
        cv2.putText(frame, f'{ID} {conf}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0))

        # Check if the vehicle crosses the line
        if LINE[0] < cx < LINE[2] and LINE[1] < cy < LINE[1] + 20:  # Define ROI/boundary
            if ID not in totalCount:
                totalCount.append(ID)

    # Draw the line and display the total count of vehicles
    cv2.line(frame, (LINE[0], LINE[1]), (LINE[2], LINE[3]), [0, 0, 255], 4)
    totalCars = len(totalCount)
    cv2.putText(frame, f'Number of vehicles: {totalCars}', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write the frame to the output video
    out.write(frame)

    # Display the frame in Google Colab
    cv2_imshow(frame)

    # Add a small delay to simulate smoother video playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer
vid.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved to: {output_video_path}")